# Creating pipelines

Here we explain how to create custom pipelines in nerfactory. Pipelines are composed of two components, namely a Dataloader and a Model.

In [29]:
# HIDDEN
%load_ext autoreload
%autoreload 2

In [30]:
from typing import Dict
import torch

from nerfactory.data.dataloader import Dataloader
from nerfactory.graphs.base import Graph
from nerfactory.pipeline.base import Pipeline

from nerfactory.graphs.instant_ngp import NGPGraph
from nerfactory.cameras.rays import RayBundle

class SemanticNGPGraph(Graph):
    """An instant ngp graph modified slightly to output semantics."""

    def __init__(self):
        self.ngp_graph = NGPGraph()

    def get_outputs(self, ray_bundle: RayBundle) -> Dict[str, torch.Tensor]:
        outputs = self.ngp_graph.forward(ray_bundle)
        outputs["semantics"] = torch.rand_like(outputs["rgb"])
        return outputs

class CustomPipeline(Pipeline):
    """The Instant NGP pipeline."""

    dataloader: Dataloader
    model: Graph

    def get_train_loss_dict(self):
        rays, batch = self.dataloader_train_iter.next()
        accumulated_color, _, _, mask = self.model(rays, batch)
        masked_batch = get_masked_dict(batch, mask)
        loss_dict = self.model.get_loss_dict(accumulated_color, masked_batch, mask)
        return loss_dict

    def get_eval_loss_dict(self):
        rays, batch = self.dataloader_eval_iter.next()
        accumulated_color, _, _, mask = self.model(rays, batch)
        masked_batch = get_masked_dict(batch, mask)
        loss_dict = self.model.get_loss_dict(accumulated_color, masked_batch, mask)
        return loss_dict

    def test_image_outputs(self) -> None:
        """Log the test image outputs"""
        # camera = Camera(self.dataloader.eval_datasetinputs.camera_to_world[0])
        pass

In [35]:
dataloader = Dataloader(use_train=True, use_eval=True)
graph = SemanticNGPGraph()
pipeline = CustomPipeline(dataloader=dataloader, model=graph)

# Creating pipelines from a config

Now we show how to create a pipeline from a config, which has the following form:

```python
@dataclass
class PipelineConfig:
    """Configuration for pipeline instantiation"""

    _target_: str = MISSING
    dataloader_config: DataloaderConfig = MISSING
    graph_config: GraphConfig = MISSING
```

See [nerfactory/utils/config.py](nerfactory/utils/config.py) for more details. In this example, we will simply load from an existing configuration from [configs/graph_instant_ngp.yaml](configs/graph_instant_ngp.yaml).

In [33]:
import pprint
import hydra
hydra.core.global_hydra.GlobalHydra.instance().clear()
from hydra import compose, initialize
initialize(version_base="1.2", config_path="../configs/")
config_name = "graph_instant_ngp.yaml"
config = compose(config_name)
pipeline_config = config.pipeline
pprint.pprint(pipeline_config)

from nerfactory.pipeline.base import setup_pipeline
pipeline = setup_pipeline(pipeline_config, device="cuda")

{'data': {'use_preprocessing_cache': False, 'dataset_inputs_train': {'_target_': 'nerfactory.data.datasets.Blender', 'data_directory': 'data/blender/lego', 'alpha_color': 'white', 'downscale_factor': 1}, 'image_dataset_train': {'_target_': 'nerfactory.data.image_dataset.ImageDataset'}, 'dataloader_train': {'_target_': 'nerfactory.data.dataloader.TrainDataloader', 'image_sampler': {'_target_': 'nerfactory.data.image_sampler.CacheImageSampler', 'num_images_to_sample_from': -1}, 'pixel_sampler': {'_target_': 'nerfactory.data.pixel_sampler.PixelSampler', 'num_rays_per_batch': 8192}}, 'dataset_inputs_eval': {'_target_': 'nerfactory.data.datasets.Blender', 'data_directory': 'data/blender/lego', 'alpha_color': 'white', 'downscale_factor': 1}, 'image_dataset_eval': {'_target_': 'nerfactory.data.image_dataset.ImageDataset'}, 'dataloader_eval': {'_target_': 'nerfactory.data.dataloader.FixedIndicesEvalDataloader', 'image_indices': [0], 'num_rays_per_chunk': 8192}}, 'graph': {'_target_': 'nerfacto